In [1]:
import pandas as pd, numpy as np
import sys, os.path
sys.path.append(os.path.abspath("../.."))
from pre_processing.id_helper import *
import rank_countries_by_stunting as rbs
import pull_gbd2019_data as pgd

from db_queries import get_ids, get_population, get_outputs, get_best_model_versions
from get_draws.api import get_draws

!date
!whoami

Fri Jan 22 15:31:20 PST 2021
ndbs


In [2]:
%load_ext autoreload
%autoreload 2

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Load iron data

In [77]:
iron_summary = pd.read_csv('data/iron_data_summary_orig.csv', index_col=[0,1], header=[0,1,2])
print(iron_summary.shape)
iron_summary.head()

(26, 54)


/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


population                           Children U5                          \
measure                    Mean hemoglobin (g/L)                           
estimate                                    mean       lower       upper   
location_id location_name                                                  
168         Angola                    107.689717  105.376743  109.981788   
161         Bangladesh                110.307373  107.457940  113.003536   
201         Burkina Faso               93.536665   91.768216   95.468099   
205         CÃ´te d'Ivoire            100.452097   98.392359  102.482946   
202         Cameroon                  107.392252  105.186886  109.697868   

population                                                               \
measure                    Number with anemia                             
estimate                                 mean        lower        upper   
location_id location_name                                                 
168         Angola                2815481.095  2455263.382  3176940.675   
161         Bangladesh            6515178.776  5284863.504  7865340.130   
201         Burkina Faso          3461606.061  3300253.760  3596616.659   
205         CÃ´te d'Ivoire        3000289.435  2767446.150  3233233.531   
202         Cameroon              2328648.778  2051380.920  2589599.321   

population                                                           \
measure                    Prevalence of anemia                       
estimate                                   mean     lower     upper   
location_id location_name                                             
168         Angola                     0.546998  0.477014  0.617223   
161         Bangladesh                 0.473972  0.384468  0.572195   
201         Burkina Faso               0.855783  0.815893  0.889161   
205         CÃ´te d'Ivoire             0.747829  0.689792  0.805891   
202         Cameroon                   0.553503  0.487599  0.615529   

population                                                       ...  \
measure                    Prevalence of iron deficiency anemia  ...   
estimate                                                   mean  ...   
location_id location_name                                        ...   
168         Angola                                     0.451482  ...   
161         Bangladesh                                 0.425066  ...   
201         Burkina Faso                               0.670929  ...   
205         CÃ´te d'Ivoire                             0.576316  ...   
202         Cameroon                                   0.432587  ...   

population                                                 WRA  \
measure                    DALYs due to iron deficiency anemia   
estimate                                                 upper   
location_id location_name                                        
168         Angola                                 80344.39769   
161         Bangladesh                            583432.27660   
201         Burkina Faso                           93490.47022   
205         CÃ´te d'Ivoire                         92607.78108   
202         Cameroon                               83139.18740   

population                                                                                   \
measure                    Maternal disorder burden attributable to iron deficiency (count)   
estimate                                                                               mean   
location_id location_name                                                                     
168         Angola                                                24669.99765                 
161         Bangladesh                                            70613.20350                 
201         Burkina Faso                                          30578.69494                 
205         CÃ´te d'Ivoire                                        33637.19879             

# Fix unicode character in Cote d'Ivoire

## First make sure location names line up

In [78]:
iron_summary2 = (iron_summary
                 .reset_index('location_name')
                 .rename(columns={'location_name': 'location_name_old'})
                )
locations = ids_to_names('location', *iron_summary.index.get_level_values('location_id'))
locations = locations.reindex(iron_summary2.index)
iron_summary2.insert(0, 'location_name', locations)
iron_summary2.set_index('location_name', append=True, inplace=True)
iron_summary2

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


population                                                   location_name_old  \
measure                                                                          
estimate                                                                         
location_id location_name                                                        
168         Angola                                                      Angola   
161         Bangladesh                                              Bangladesh   
201         Burkina Faso                                          Burkina Faso   
205         Côte d'Ivoire                                       CÃ´te d'Ivoire   
202         Cameroon                                                  Cameroon   
6           China                                                        China   
171         Democratic Republic of the Congo  Democratic Republic of the Congo   
141         Egypt                                                        Egypt   
179         Ethiopia                                                  Ethiopia   
207         Ghana                                                        Ghana   
1           Global                                                      Global   
163         India                                                        India   
11          Indonesia                                                Indonesia   
180         Kenya                                                        Kenya   
181         Madagascar                                              Madagascar   
184         Mozambique                                              Mozambique   
15          Myanmar                                                    Myanmar   
164         Nepal                                                        Nepal   
213         Niger                                                        Niger   
214         Nigeria                                                    Nigeria   
165         Pakistan                                                  Pakistan   
196         South Africa                                          South Africa   
522         Sudan                                                        Sudan   
190         Uganda                                                      Uganda   
189         United Republic of Tanzania            United Republic of Tanzania   
20          Viet Nam                                                  Viet Nam   

population                                             Children U5  \
measure                                      Mean hemoglobin (g/L)   
estimate                                                      mean   
location_id location_name                                            
168         Angola                                      107.689717   
161         Bangladesh                                  110.307373   
201         Burkina Faso                                 93.536665   
205         Côte d'Ivoire                               100.452097   
202         Cameroon                                    107.392252   
6           China                                       128.593022   
171         Democratic Republic of the Congo            105.366639   
141         Egypt                                       116.496979   
179         Ethiopia                                    106.425493   
207         Ghana                                       103.297894   
1           Global                                      113.680438   
163         India                                       106.689123   
11          Indonesia                                   121.223057   
180         Kenya                                       112.436172   
181         Madagascar                                  110.649281   
184         Mozambique                                  102.526617   
15          Myanmar                                     105.411794   
164         Nepal                                       108.465575   

### Looks good, so lets' drop the old location names

In [79]:
iron_summary2.drop(columns='location_name_old', level=0, inplace=True)
iron_summary2

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


population                                             Children U5  \
measure                                      Mean hemoglobin (g/L)   
estimate                                                      mean   
location_id location_name                                            
168         Angola                                      107.689717   
161         Bangladesh                                  110.307373   
201         Burkina Faso                                 93.536665   
205         Côte d'Ivoire                               100.452097   
202         Cameroon                                    107.392252   
6           China                                       128.593022   
171         Democratic Republic of the Congo            105.366639   
141         Egypt                                       116.496979   
179         Ethiopia                                    106.425493   
207         Ghana                                       103.297894   
1           Global                                      113.680438   
163         India                                       106.689123   
11          Indonesia                                   121.223057   
180         Kenya                                       112.436172   
181         Madagascar                                  110.649281   
184         Mozambique                                  102.526617   
15          Myanmar                                     105.411794   
164         Nepal                                       108.465575   
213         Niger                                       101.479046   
214         Nigeria                                     100.152790   
165         Pakistan                                    110.634751   
196         South Africa                                123.267542   
522         Sudan                                       109.096853   
190         Uganda                                      109.112418   
189         United Republic of Tanzania                 106.501450   
20          Viet Nam                                    120.394288   

population                                                            \
measure                                                                
estimate                                           lower       upper   
location_id location_name                                              
168         Angola                            105.376743  109.981788   
161         Bangladesh                        107.457940  113.003536   
201         Burkina Faso                       91.768216   95.468099   
205         Côte d'Ivoire                      98.392359  102.482946   
202         Cameroon                          105.186886  109.697868   
6           China                             127.740049  129.409293   
171         Democratic Republic of the Congo  103.219266  107.485872   
141         Egypt                             114.202490  118.841999   
179         Ethiopia                          105.460935  107.370882   
207         Ghana                             100.856822  105.782141   
1           Global                            113.395944  113.987582   
163         India                             106.194938  107.187063   
11          Indonesia                         118.539064  123.740339   
180         Kenya                             111.952395  112.886144   
181         Madagascar                        108.618853  112.686471   
184         Mozambique                        100.757783  104.386462   
15          Myanmar                           103.167630  107.550940   
164         Nepal                             105.747604  111.036324   
213         Niger                              99.002071  103.836821   
214         Nigeria                            98.525898  101.607618   
165         Pakistan                          107.472944  113.943664   
196         South Africa                      120.926779  125.738562   
522         Sudan          

# Move global to end

In [80]:
iron_summary3 = pgd.move_global_to_end(iron_summary2)
iron_summary3

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


population                                             Children U5  \
measure                                      Mean hemoglobin (g/L)   
estimate                                                      mean   
location_id location_name                                            
168         Angola                                      107.689717   
161         Bangladesh                                  110.307373   
201         Burkina Faso                                 93.536665   
205         Côte d'Ivoire                               100.452097   
202         Cameroon                                    107.392252   
6           China                                       128.593022   
171         Democratic Republic of the Congo            105.366639   
141         Egypt                                       116.496979   
179         Ethiopia                                    106.425493   
207         Ghana                                       103.297894   
163         India                                       106.689123   
11          Indonesia                                   121.223057   
180         Kenya                                       112.436172   
181         Madagascar                                  110.649281   
184         Mozambique                                  102.526617   
15          Myanmar                                     105.411794   
164         Nepal                                       108.465575   
213         Niger                                       101.479046   
214         Nigeria                                     100.152790   
165         Pakistan                                    110.634751   
196         South Africa                                123.267542   
522         Sudan                                       109.096853   
190         Uganda                                      109.112418   
189         United Republic of Tanzania                 106.501450   
20          Viet Nam                                    120.394288   
1           Global                                      113.680438   

population                                                            \
measure                                                                
estimate                                           lower       upper   
location_id location_name                                              
168         Angola                            105.376743  109.981788   
161         Bangladesh                        107.457940  113.003536   
201         Burkina Faso                       91.768216   95.468099   
205         Côte d'Ivoire                      98.392359  102.482946   
202         Cameroon                          105.186886  109.697868   
6           China                             127.740049  129.409293   
171         Democratic Republic of the Congo  103.219266  107.485872   
141         Egypt                             114.202490  118.841999   
179         Ethiopia                          105.460935  107.370882   
207         Ghana                             100.856822  105.782141   
163         India                             106.194938  107.187063   
11          Indonesia                         118.539064  123.740339   
180         Kenya                             111.952395  112.886144   
181         Madagascar                        108.618853  112.686471   
184         Mozambique                        100.757783  104.386462   
15          Myanmar                           103.167630  107.550940   
164         Nepal                             105.747604  111.036324   
213         Niger                              99.002071  103.836821   
214         Nigeria                            98.525898  101.607618   
165         Pakistan                          107.472944  113.943664   
196         South Africa                      120.926779  125.738562   
522         Sudan                             106.922764  111.226004   
190         Uganda         

# Save cleaned file and verify

In [81]:
iron_summary3.to_csv('data/iron_data_summary.csv')
pd.read_csv('data/iron_data_summary.csv', index_col=[0,1], header=[0,1,2])

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


population                                             Children U5  \
measure                                      Mean hemoglobin (g/L)   
estimate                                                      mean   
location_id location_name                                            
168         Angola                                      107.689717   
161         Bangladesh                                  110.307373   
201         Burkina Faso                                 93.536665   
205         Côte d'Ivoire                               100.452097   
202         Cameroon                                    107.392252   
6           China                                       128.593022   
171         Democratic Republic of the Congo            105.366639   
141         Egypt                                       116.496979   
179         Ethiopia                                    106.425493   
207         Ghana                                       103.297894   
163         India                                       106.689123   
11          Indonesia                                   121.223057   
180         Kenya                                       112.436172   
181         Madagascar                                  110.649281   
184         Mozambique                                  102.526617   
15          Myanmar                                     105.411794   
164         Nepal                                       108.465575   
213         Niger                                       101.479046   
214         Nigeria                                     100.152790   
165         Pakistan                                    110.634751   
196         South Africa                                123.267542   
522         Sudan                                       109.096853   
190         Uganda                                      109.112418   
189         United Republic of Tanzania                 106.501450   
20          Viet Nam                                    120.394288   
1           Global                                      113.680438   

population                                                            \
measure                                                                
estimate                                           lower       upper   
location_id location_name                                              
168         Angola                            105.376743  109.981788   
161         Bangladesh                        107.457940  113.003536   
201         Burkina Faso                       91.768216   95.468099   
205         Côte d'Ivoire                      98.392359  102.482946   
202         Cameroon                          105.186886  109.697868   
6           China                             127.740049  129.409293   
171         Democratic Republic of the Congo  103.219266  107.485872   
141         Egypt                             114.202490  118.841999   
179         Ethiopia                          105.460935  107.370882   
207         Ghana                             100.856822  105.782141   
163         India                             106.194938  107.187063   
11          Indonesia                         118.539064  123.740339   
180         Kenya                             111.952395  112.886144   
181         Madagascar                        108.618853  112.686470   
184         Mozambique                        100.757783  104.386462   
15          Myanmar                           103.167630  107.550940   
164         Nepal                             105.747604  111.036324   
213         Niger                              99.002071  103.836821   
214         Nigeria                            98.525898  101.607618   
165         Pakistan                          107.472944  113.943664   
196         South Africa                      120.926779  125.738562   
522         Sudan                             106.922764  111.226004   
190         Uganda         

# Stack population index

In [82]:
iron_summary3 = iron_summary3.stack('population')
iron_summary3.head(10)

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


measure                               DALYs attributable to iron deficiency  \
estimate                                                              lower   
location_id location_name population                                          
168         Angola        Children U5                           39224.83481   
                          WRA                                   49816.35179   
161         Bangladesh    Children U5                           80232.90776   
                          WRA                                  302307.22720   
201         Burkina Faso  Children U5                           79799.50206   
                          WRA                                   55996.16132   
205         Côte d'Ivoire Children U5                           47796.68964   
                          WRA                                   62243.99601   
202         Cameroon      Children U5                           32807.17401   
                          WRA                                   58090.24751   

measure                                                            \
estimate                                       mean         upper   
location_id location_name population                                
168         Angola        Children U5   65575.59769  101718.60180   
                          WRA           78990.65521  113520.52160   
161         Bangladesh    Children U5  141131.26830  227198.61480   
                          WRA          455148.09410  662410.34670   
201         Burkina Faso  Children U5  120326.60040  176129.96580   
                          WRA           89642.04557  133214.68160   
205         Côte d'Ivoire Children U5   74680.44881  112824.05610   
                          WRA           96560.30634  136533.08000   
202         Cameroon      Children U5   52508.43941   80169.42294   
                          WRA           92894.46562  133170.65490   

measure                               DALYs attributable to iron deficiency per person-year  \
estimate                                                                              lower   
location_id location_name population                                                          
168         Angola        Children U5                                           0.007621      
                          WRA                                                   0.005221      
161         Bangladesh    Children U5                                           0.005837      
                          WRA                                                   0.005377      
201         Burkina Faso  Children U5                                           0.019728      
                          WRA                                                   0.007808      
205         Côte d'Ivoire Children U5                                           0.011913      
                          WRA                                                   0.007619      
202         Cameroon      Children U5                                           0.007798      
                          WRA                                                   0.006077      

measure                                                   DALYs due to anemia  \
estimate                                   mean     upper               lower   
location_id location_name population                                            
168         Angola        Children U5  0.012740  0.019762         51961.88640   
                          WRA          0.008278  0.011897         50978.73078   
161         Bangladesh    Children U5  0.010267  0.016528         91183.58386   
                          WRA          0.008095  0.011781        283743.36230   
201         Burkina Faso  Children U5  0.029747  0.043543        110984.31520   
                          WRA          0.012499  0.018574         69252.14252   
205         Côte d'Ivoire Children U5  0.018614  0.028122         73334.37475   
                          WR

# Drop unneeded columns

In [83]:
iron_summary3.columns.get_level_values('measure').unique()

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index(['DALYs attributable to iron deficiency',
       'DALYs attributable to iron deficiency per person-year',
       'DALYs due to anemia', 'DALYs due to iron deficiency anemia',
       'Maternal disorder burden attributable to iron deficiency (count)',
       'Mean hemoglobin (g/L)', 'Number with anemia',
       'Number with iron deficiency anemia', 'Prevalence of anemia',
       'Prevalence of iron deficiency anemia'],
      dtype='object', name='measure')

In [84]:
iron_summary3 = iron_summary3.drop(
    columns=[
        'DALYs due to anemia',
        'DALYs due to iron deficiency anemia',
        'Maternal disorder burden attributable to iron deficiency (count)',
        'Number with anemia',
    ])
iron_summary3.head()

measure                               DALYs attributable to iron deficiency  \
estimate                                                              lower   
location_id location_name population                                          
168         Angola        Children U5                           39224.83481   
                          WRA                                   49816.35179   
161         Bangladesh    Children U5                           80232.90776   
                          WRA                                  302307.22720   
201         Burkina Faso  Children U5                           79799.50206   

measure                                                           \
estimate                                       mean        upper   
location_id location_name population                               
168         Angola        Children U5   65575.59769  101718.6018   
                          WRA           78990.65521  113520.5216   
161         Bangladesh    Children U5  141131.26830  227198.6148   
                          WRA          455148.09410  662410.3467   
201         Burkina Faso  Children U5  120326.60040  176129.9658   

measure                               DALYs attributable to iron deficiency per person-year  \
estimate                                                                              lower   
location_id location_name population                                                          
168         Angola        Children U5                                           0.007621      
                          WRA                                                   0.005221      
161         Bangladesh    Children U5                                           0.005837      
                          WRA                                                   0.005377      
201         Burkina Faso  Children U5                                           0.019728      

measure                                                    \
estimate                                   mean     upper   
location_id location_name population                        
168         Angola        Children U5  0.012740  0.019762   
                          WRA          0.008278  0.011897   
161         Bangladesh    Children U5  0.010267  0.016528   
                          WRA          0.008095  0.011781   
201         Burkina Faso  Children U5  0.029747  0.043543   

measure                               Mean hemoglobin (g/L)              \
estimate                                              lower        mean   
location_id location_name population                                      
168         Angola        Children U5            105.376743  107.689717   
                          WRA                    121.124472  123.299716   
161         Bangladesh    Children U5            107.457940  110.307373   
                          WRA                    122.323484  124.194877   
201         Burkina Faso  Children U5             91.768216   93.536665   

measure                                            \
estimate                                    upper   
location_id location_name population                
168         Angola        Children U5  109.981788   
                          WRA          125.444390   
161         Bangladesh    Children U5  113.003536   
                          WRA          126.198742   
201         Burkina Faso  Children U5   95.468099   

measure                               Number with iron deficiency anemia  \
estimate                                                           lower   
location_id location_name population                                       
168         Angola        Children U5                       2.455263e+06   
                          WRA                               2.528641e+06   
161         Bangladesh    Children U5                       5.284864e+06   
                          WRA                               1.538142e+07   
201         Bu

In [85]:
iron_summary3 = iron_summary3[
    ['Mean hemoglobin (g/L)',
     'Prevalence of anemia',
     'Prevalence of iron deficiency anemia',
     'Number with iron deficiency anemia',
     'DALYs attributable to iron deficiency',
     'DALYs attributable to iron deficiency per person-year'
    ]]
iron_summary3.head()

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


measure                               Mean hemoglobin (g/L)              \
estimate                                              lower        mean   
location_id location_name population                                      
168         Angola        Children U5            105.376743  107.689717   
                          WRA                    121.124472  123.299716   
161         Bangladesh    Children U5            107.457940  110.307373   
                          WRA                    122.323484  124.194877   
201         Burkina Faso  Children U5             91.768216   93.536665   

measure                                           Prevalence of anemia  \
estimate                                    upper                lower   
location_id location_name population                                     
168         Angola        Children U5  109.981788             0.477014   
                          WRA          125.444390             0.265702   
161         Bangladesh    Children U5  113.003536             0.384468   
                          WRA          126.198742             0.274773   
201         Burkina Faso  Children U5   95.468099             0.815893   

measure                                                    \
estimate                                   mean     upper   
location_id location_name population                        
168         Angola        Children U5  0.546998  0.617223   
                          WRA          0.313496  0.364626   
161         Bangladesh    Children U5  0.473972  0.572195   
                          WRA          0.321329  0.367614   
201         Burkina Faso  Children U5  0.855783  0.889161   

measure                               Prevalence of iron deficiency anemia  \
estimate                                                             lower   
location_id location_name population                                         
168         Angola        Children U5                             0.387393   
                          WRA                                     0.149143   
161         Bangladesh    Children U5                             0.341107   
                          WRA                                     0.213748   
201         Burkina Faso  Children U5                             0.606678   

measure                                                    \
estimate                                   mean     upper   
location_id location_name population                        
168         Angola        Children U5  0.451482  0.518286   
                          WRA          0.179470  0.212595   
161         Bangladesh    Children U5  0.425066  0.516298   
                          WRA          0.255636  0.299256   
201         Burkina Faso  Children U5  0.670929  0.751787   

measure                               Number with iron deficiency anemia  \
estimate                                                           lower   
location_id location_name population                                       
168         Angola        Children U5                       2.455263e+06   
                          WRA                               2.528641e+06   
161         Bangladesh    Children U5                       5.284864e+06   
                          WRA                               1.538142e+07   
201         Burkina Faso  Children U5                       3.300254e+06   

measure                                                            \
estimate                                       mean         upper   
location_id location_name population                                
168         Angola        Children U5  2.815481e+06  3.176941e+06   
                          WRA          2.988579e+06  3.479227e+06   
161         Bangladesh    Children U5  6.515179e+06  7.865340e+06   
                          WRA          1.805022e+07  2.066993e+07   
201         Burkina Faso  Children U5  3.461606e+06  3.596617e+06   

measure                               DAL

# Format data summary and save

In [89]:
iron_formatted = pgd.format_data_summary(
    iron_summary3,
    daly_count_units=1000,
    daly_rate_multiplier=100_000,
    save_filename='data/iron_data_formatted.csv'
)
iron_formatted

/ihme/homes/ndbs/miniconda3/envs/my_gbd_2019/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mean hemoglobin (g/L)  \
location_id location_name                    population                          
168         Angola                           Children U5       108\n(105, 110)   
                                             WRA               123\n(121, 125)   
161         Bangladesh                       Children U5       110\n(107, 113)   
                                             WRA               124\n(122, 126)   
201         Burkina Faso                     Children U5          94\n(92, 95)   
                                             WRA               119\n(117, 121)   
205         Côte d'Ivoire                    Children U5        100\n(98, 102)   
                                             WRA               119\n(118, 121)   
202         Cameroon                         Children U5       107\n(105, 110)   
                                             WRA               124\n(122, 126)   
6           China                            Children U5       129\n(128, 129)   
                                             WRA               136\n(136, 137)   
171         Democratic Republic of the Congo Children U5       105\n(103, 107)   
                                             WRA               121\n(119, 122)   
141         Egypt                            Children U5       116\n(114, 119)   
                                             WRA               127\n(125, 129)   
179         Ethiopia                         Children U5       106\n(105, 107)   
                                             WRA               130\n(129, 131)   
207         Ghana                            Children U5       103\n(101, 106)   
                                             WRA               120\n(118, 122)   
163         India                            Children U5       107\n(106, 107)   
                                             WRA               118\n(117, 118)   
11          Indonesia                        Children U5       121\n(119, 124)   
                                             WRA               127\n(126, 129)   
180         Kenya                            Children U5       112\n(112, 113)   
                                             WRA               129\n(129, 130)   
181         Madagascar                       Children U5       111\n(109, 113)   
                                             WRA               124\n(122, 126)   
184         Mozambique                       Children U5       103\n(101, 104)   
                                             WRA               122\n(120, 123)   
15          Myanmar                          Children U5       105\n(103, 108)   
                                             WRA               122\n(121, 124)   
164         Nepal                            Children U5       108\n(106, 111)   
                                             WRA               123\n(121, 125)   
213         Niger                            Children U5        101\n(99, 104)   
                                             WRA               119\n(117, 121)   
214         Nigeria                          Children U5        100\n(99, 102)   
                                             WRA               119\n(117, 120)   
165         Pakistan                         Children U5       111\n(107, 114)   
                                             WRA               119\n(118, 121)   
196         South Africa                     Children U5       123\n(121, 126)   
                                             WRA               129\n(127, 130)   
522         Sudan                            Children U5       109\n(107, 111)   
                                             WRA               125\n(123, 127)   
190         Uganda                           Children U5       109\n(107, 111)   
                                             WRA               126\n(124, 129)   
189         United Republic of Tanzania      Children U5       107\n(104, 109)   
                                       